In [ ]:
import sys
import os
os.chdir("..")
import pandas as pd

In [ ]:
from src.modeling.train_model import *

In [ ]:
# Load dữ liệu
X_train, y_train, X_val, y_val, X_test, y_test = load_data()

Ở đây chúng tôi sẽ train trên 5 mô hình để tìm ra mô hình mang lại độ chính xác lớn nhất để sử dụng gồm: 
- LightGBM
- XGBoost
- RandomForest
- GradientBoost
- LogisticRegression

In [ ]:
# Khởi tạo các preprocessors cho mô hình tree và linear
pre_tree, pre_lin = build_preprocessors(X_train)
pre_tree.fit(X_train)
X_val_tree = pre_tree.transform(X_val)
cls_weight_dict, sample_weight_train = make_class_weight(y_train) # normalize frequency các class đầu ra

Chúng tôi sử dụng GridSearchCV để chạy nhiều lần lặp giúp mô hình tìm được parameter phù hợp và tối ưu nhất

In [ ]:
# Xác định các model và hyperparameters
models = {
    "LightGBM": (
        LGBMClassifier(objective='multiclass', metric='multi_logloss', 
                        class_weight = cls_weight_dict, device='gpu', random_state=42,
                        callbacks=[lgb_early_stop(stopping_rounds=50, first_metric_only=True, verbose=False)]),
        { "model__num_leaves":       randint(31, 512),
            "model__max_depth":        [-1, 7, 11, 15, 19],
            "model__learning_rate":    uniform(0.01, 0.29),
            "model__n_estimators":     randint(200, 801),
            "model__min_child_samples":randint(10, 200) },
        pre_tree,
        {"model__eval_set":[(X_val_tree,y_val)]}
    ),
    "XGBoost": (
        XGBClassifier(objective='multi:softprob',
                        tree_method='hist', device = 'cuda', eval_metric='mlogloss',
                        random_state=42, callbacks=[xgb_early_stop(rounds=50, metric_name='mlogloss', data_name ='validation_0', save_best=False)]),
        { "model__max_depth":        randint(3, 12),
            "model__learning_rate":    uniform(0.01, 0.29),
            "model__n_estimators":     randint(200, 800),
            "model__subsample":        uniform(0.6, 0.4),
            "model__colsample_bytree": uniform(0.6, 0.4) },
        pre_tree,
        {"model__eval_set":[(X_val_tree,y_val)]}
    ),
    "RandomForest": (
        RandomForestClassifier(class_weight=cls_weight_dict,
                                n_jobs=-1, random_state=42),
        { "model__n_estimators":     randint(200,800),
            "model__max_depth":        [None, 10, 20, 30],
            "model__min_samples_split":randint(2,20),
            "model__min_samples_leaf": randint(1,10) },
        pre_tree,
        {}
    ),
    "GradientBoost": (
        GradientBoostingClassifier(random_state=42),
        { "model__learning_rate":    uniform(0.01,0.29),
            "model__n_estimators":     randint(100,600),
            "model__max_depth":        randint(2,7),
            "model__subsample":        uniform(0.5,0.5) },
        pre_tree,
        {}
    ),
    "LogisticRegression": (
        LogisticRegression(max_iter=4000, multi_class='multinomial',
                            n_jobs=-1),
        { "model__C":        uniform(0.01, 100),
            "model__penalty":  ['l2','none'],
            "model__solver":   ['lbfgs','saga'] },
        pre_lin,
        {}
    )
}

Và chúng tôi thêm Randomized search CV để giảm số lần lặp train model nhưng vẫn giữ độ chính xác tốt

In [ ]:
# Train, validate các model, và lưu lại model tốt nhất trên tập val
tscv   = TimeSeriesSplit(n_splits=3)
best_pipe, best_score, best_name = None, -1, ""

os.makedirs("models", exist_ok=True)

for name,(est,dist,prep,fit_kw) in models.items():
    n_iter = 30 if name in {"RandomForest", "GradientBoost"} else 50
    print(f"\n▶▶ {name}: Randomized search ({n_iter} configs, 3-fold)")
    
    pipe = Pipeline([('prep', prep), ('model', est)])
    search = RandomizedSearchCV(
        pipe, dist, n_iter=n_iter, cv=tscv,
        scoring='accuracy', n_jobs=-1, random_state=42, verbose=1, refit=True
    )
    
    tic = time.time()
    search.fit(X_train, y_train, **fit_kw)
    toc = time.time()
    print(f"↳ Done in {(toc-tic)/60:.1f} min — best Acc={search.best_score_:.4f}")

    # Lưu hyperparameters tốt nhất
    with open(f"models/{name}_best.json", "w") as fp:
        json.dump(search.best_params_, fp, indent=2, default=str)

    # In top-5 cấu hình tốt nhất
    top5 = (pd.DataFrame(search.cv_results_)
            .sort_values("rank_test_score")
            .head(5)[["mean_test_score", "params"]])
    print(top5.to_string(index=False))
    
    # Đánh giá trên tập val (season 2014/2015)
    y_pred = search.predict(X_val)
    acc = accuracy_score(y_val, y_pred)
    print(classification_report(y_val, y_pred, digits=4))

    # Cập nhật best
    if acc > best_score:
        best_score, best_pipe, best_name = acc, search.best_estimator_, name

    # Lưu checkpoint (chứa cả pipeline + preprocessors)
    joblib.dump(search.best_estimator_, f"models/{name}_best.pkl")

In [ ]:
# Evaluate trên tập test 2015/2016
if best_pipe is None:
    raise RuntimeError("Không có mô hình nào huấn luyện thành công!")

print(f"\nBest model = {best_name}  (val Acc={best_score:.4f})")
y_test_pred = best_pipe.predict(X_test)
print(classification_report(y_test, y_test_pred, digits=4))